In [1]:
import os
import sys

sys.path.append( os.path.abspath("../../panda") )

from time import time, sleep                                                                      
import struct                                                                                     
import argparse                                                                                   
import contextlib                                                                                 
import io                                                                                         
                                                                                                  
from prompt_toolkit import PromptSession                                                          
from prompt_toolkit.history import FileHistory                                                    
from panda import Panda  

def init(bus, speed_kbps, silent=False):                                                          
   f = io.StringIO()                                                                               
   with contextlib.redirect_stdout(f):                                                             
     panda = Panda()                                                                               
   if DEBUG or not silent: print(f.getvalue())                                                     
   if bus == -1:                                                                                   
     bus = 1 if panda.has_obd() else 0                                                             
   if DEBUG: print(f'--- BUS: {bus}')                                                              
   if speed_kbps:                                                                                  
     if DEBUG: print(f'--- SPEED: {speed_kbps} kbps')                                              
     panda.set_can_speed_kbps(bus, speed_kbps)                                                     
   panda.set_safety_mode(Panda.SAFETY_ALLOUTPUT)                                                   
   return panda, bus    

DEBUG = 1
CAN_BUS = 0
SPEED_KBPS = 500

bus = CAN_BUS
is_tty = sys.stdin.isatty() 
panda, bus = init(CAN_BUS, SPEED_KBPS, silent=not is_tty)

panda.can_clear(bus)    #flash TX
panda.can_clear(0xFFFF) #flash RX
start_time = time()
i = 0
while time() - start_time < 1:
    msgs = panda.can_recv()
    # can_id , ts(us), data, bus 
    for rx_addr, ts, dat, src in msgs:
        i += 1
        if DEBUG: print(f'\n---> RX {src} |{ts}| {hex(rx_addr)}: 0x{dat.hex()} {i}')
                        
print(panda,bus)

panda.close()

opening device 410020000651363038363036 0xddcc
connected

--- BUS: 0
--- SPEED: 500 kbps
<panda.Panda object at 0x7fb6c03d3490> 0


In [4]:
#drivers/can.h
# // assign CAN numbering
# // bus num: Can bus number on ODB connector. Sent to/from USB
# //    Min: 0; Max: 127; Bit 7 marks message as receipt (bus 129 is receipt for but 1)
# // cans: Look up MCU can interface from bus number
# // can number: numeric lookup for MCU CAN interfaces (0 = CAN1, 1 = CAN2, etc);
# // bus_lookup: Translates from 'can number' to 'bus number'.
# // can_num_lookup: Translates from 'bus number' to 'can number'.
# // can_forwarding: Given a bus num, lookup bus num to forward to. -1 means no forward.

# // Panda:       Bus 0=CAN1   Bus 1=CAN2   Bus 2=CAN3
# CAN_TypeDef *cans[] = {CAN1, CAN2, CAN3};
# uint8_t bus_lookup[] = {0,1,2};
# uint8_t can_num_lookup[] = {0,1,2,-1};
# int8_t can_forwarding[] = {-1,-1,-1,-1};
# uint32_t can_speed[] = {5000, 5000, 5000, 333};
# #define CAN_MAX 3U

# #define CANIF_FROM_CAN_NUM(num) (cans[num])
# #define CAN_NUM_FROM_CANIF(CAN) ((CAN)==CAN1 ? 0 : ((CAN) == CAN2 ? 1 : 2))
# #define BUS_NUM_FROM_CAN_NUM(num) (bus_lookup[num])
# #define CAN_NUM_FROM_BUS_NUM(num) (can_num_lookup[num])

#@can_rx()
#can_forwarding[] = {-1,-1,-1,-1};
#void can_set_forwarding(int from, int to) {
#   can_forwarding[from] = to;
# }
#
#can_forwaring[0] = 1     rx from bus0 and tx to bu1

# void process_can(uint8_t can_number) {
#   if (can_number != 0xffU) {

#     ENTER_CRITICAL();

#     CAN_TypeDef *CAN = CANIF_FROM_CAN_NUM(can_number);
#     uint8_t bus_number = BUS_NUM_FROM_CAN_NUM(can_number);

#     // check for empty mailbox
#     CAN_FIFOMailBox_TypeDef to_send;
#     if ((CAN->TSR & CAN_TSR_TME0) == CAN_TSR_TME0) {
#       // add successfully transmitted message to my fifo
#       if ((CAN->TSR & CAN_TSR_RQCP0) == CAN_TSR_RQCP0) {
#         can_txd_cnt += 1;

#         if ((CAN->TSR & CAN_TSR_TXOK0) == CAN_TSR_TXOK0) {
#           CAN_FIFOMailBox_TypeDef to_push;
#           to_push.RIR = CAN->sTxMailBox[0].TIR;
#           to_push.RDTR = (CAN->sTxMailBox[0].TDTR & 0xFFFF000FU) | ((CAN_BUS_RET_FLAG | bus_number) << 4);  //!!!//
#           to_push.RDLR = CAN->sTxMailBox[0].TDLR;
#           to_push.RDHR = CAN->sTxMailBox[0].TDHR;
#           can_send_errs += can_push(&can_rx_q, &to_push) ? 0U : 1U;
#         }

#         if ((CAN->TSR & CAN_TSR_TERR0) == CAN_TSR_TERR0) {
#           #ifdef DEBUG
#             puts("CAN TX ERROR!\n");
#           #endif
#         }

#         if ((CAN->TSR & CAN_TSR_ALST0) == CAN_TSR_ALST0) {
#           #ifdef DEBUG
#             puts("CAN TX ARBITRATION LOST!\n");
#           #endif
#         }

#         // clear interrupt
#         // careful, this can also be cleared by requesting a transmission
#         CAN->TSR |= CAN_TSR_RQCP0;
#       }

#       if (can_pop(can_queues[bus_number], &to_send)) {
#         can_tx_cnt += 1;
#         // only send if we have received a packet
#         CAN->sTxMailBox[0].TDLR = to_send.RDLR;
#         CAN->sTxMailBox[0].TDHR = to_send.RDHR;
#         CAN->sTxMailBox[0].TDTR = to_send.RDTR;
#         CAN->sTxMailBox[0].TIR = to_send.RIR;

#         if (can_tx_check_min_slots_free(MAX_CAN_MSGS_PER_BULK_TRANSFER)) {
#           usb_outep3_resume_if_paused();
#         }
#       }
#     }

#     EXIT_CRITICAL();
#   }
# }
#
# Send message and push can_rx_q with setting bit7 to 1

import os
import sys

sys.path.append( os.path.abspath("../../panda") )

print(sys.path)
from time import time, sleep                                                                      
import struct                                                                                     
import argparse                                                                                   
import contextlib                                                                                 
import io                                                                                         
                                                                                                  
from prompt_toolkit import PromptSession                                                          
from prompt_toolkit.history import FileHistory                                                    
from panda import Panda


panda = Panda()

speed_kbps = 500

for bus in range(3):
    panda.set_can_speed_kbps(bus, speed_kbps) 
    panda.set_can_forwarding(bus, -1)   #disable all forwarding
    
panda.set_safety_mode(Panda.SAFETY_CANGW) 
#panda.set_safety_mode(200) 


panda.set_can_forwarding(0,2)
panda.set_can_forwarding(2,0)

panda.close()


['/home/ding/fun/hackcar/candbg/panda/notebook', '/home/ding/.pyenv/versions/3.8.2/lib/python38.zip', '/home/ding/.pyenv/versions/3.8.2/lib/python3.8', '/home/ding/.pyenv/versions/3.8.2/lib/python3.8/lib-dynload', '', '/home/ding/.pyenv/versions/3.8.2/lib/python3.8/site-packages', '/home/ding/.pyenv/versions/3.8.2/lib/python3.8/site-packages/carla-0.9.7-py3.5-linux-x86_64.egg', '/home/ding/.pyenv/versions/3.8.2/lib/python3.8/site-packages/IPython/extensions', '/home/ding/.ipython', '/home/ding/fun/hackcar/candbg/panda', '/home/ding/fun/hackcar/candbg/panda', '/home/ding/fun/hackcar/candbg/panda', '/home/ding/fun/hackcar/candbg/panda']
opening device 410020000651363038363036 0xddcc
exception LIBUSB_ERROR_BUSY [-6]
connected


Traceback (most recent call last):
  File "/home/ding/.pyenv/versions/3.8.2/lib/python3.8/site-packages/panda/__init__.py", line 191, in connect
    self._handle.claimInterface(0)
  File "/home/ding/.pyenv/versions/3.8.2/lib/python3.8/site-packages/usb1/__init__.py", line 1275, in claimInterface
    mayRaiseUSBError(
  File "/home/ding/.pyenv/versions/3.8.2/lib/python3.8/site-packages/usb1/__init__.py", line 133, in mayRaiseUSBError
    __raiseUSBError(value)
  File "/home/ding/.pyenv/versions/3.8.2/lib/python3.8/site-packages/usb1/__init__.py", line 125, in raiseUSBError
    raise __STATUS_TO_EXCEPTION_DICT.get(value, __USBError)(value)
usb1.USBErrorBusy: LIBUSB_ERROR_BUSY [-6]


AttributeError: type object 'Panda' has no attribute 'SAFETY_CANGW'

In [35]:
for bus in range(3):
    print(bus)

0
1
2


In [1]:
import sys
sys.path.append("/home/ding/openpilot-master-tools")

import sys
import zmq
import time
from collections import defaultdict, OrderedDict

#from selfdrive.boardd.boardd import can_list_to_can_capnp
#from selfdrive.car.toyota.toyotacan import make_can_msg
import cereal.messaging as messaging
from cereal.services import service_list
